# Classifiers and training

Import required libraries.

In [1]:
import time
import scipy.io
import mne
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import antropy as ant
import EntropyHub as EH
from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report as clfrep
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

Load calculated entropies from files.

In [2]:
AE_normal_matrix = np.load("data/AE_normal_matrix.npy")
PE_normal_matrix = np.load("data/PE_normal_matrix.npy")
SE_normal_matrix = np.load("data/SE_normal_matrix.npy")
FE_normal_matrix = np.load("data/FE_normal_matrix.npy")

AE_fatigue_matrix = np.load("data/AE_fatigue_matrix.npy")
PE_fatigue_matrix = np.load("data/PE_fatigue_matrix.npy")
SE_fatigue_matrix = np.load("data/SE_fatigue_matrix.npy")
FE_fatigue_matrix = np.load("data/FE_fatigue_matrix.npy")

ENTROPIES = [
    [AE_normal_matrix, AE_fatigue_matrix], 
    [PE_normal_matrix, PE_fatigue_matrix], 
    [SE_normal_matrix, SE_fatigue_matrix], 
    [FE_normal_matrix, FE_fatigue_matrix]
]

num_entropies = 4
num_people = 12
num_epochs = 300
num_channels = 30

Clean NaN and infinity values.

In [3]:
for i in range(len(ENTROPIES)):
    for j in range(len(ENTROPIES[i])):
        np.nan_to_num(ENTROPIES[i][j], copy=False, nan=0.0, posinf=0.0, neginf=0.0)
        np.nan_to_num(ENTROPIES[i][j], copy=False, nan=0.0, posinf=0.0, neginf=0.0)

Generate X matrix. `train_index` contains indexes of people that will be used in dataset.

In [4]:
def get_all_X(train_index):
    matrix_X = np.zeros(shape=(num_entropies, 2*num_epochs, num_channels))

    for person in train_index:
        for entropy in range(num_entropies):
            normal_X = np.zeros(shape=(num_epochs, num_channels))
            for ch in range(num_channels):
                for epoch in range(num_epochs):
                    normal_X[epoch][ch] = (ENTROPIES[entropy][0][person][epoch][ch][0])
            fatigue_X = np.zeros(shape=(num_epochs, num_channels))
            for ch in range(num_channels):
                for epoch in range(num_epochs):
                    fatigue_X[epoch][ch] = (ENTROPIES[entropy][1][person][epoch][ch][0])
                  
            # scale
            scaler = MinMaxScaler((-1, 1))
            normal_scaled_X = scaler.fit_transform(normal_X)
            fatigue_scaled_X = scaler.fit_transform(fatigue_X)
            
            X = np.concatenate((normal_scaled_X, fatigue_scaled_X), axis=0)
            matrix_X[entropy] = X
        X_one_person = np.concatenate((matrix_X[0], matrix_X[1], matrix_X[2], matrix_X[3]), axis=1)
        if person == train_index[0]:
            matrix_all_X = X_one_person
        else:
            matrix_all_X = np.concatenate((matrix_all_X, X_one_person), axis=0)
        
    return matrix_all_X

Generate y label matrix. ```train_index``` contains indexes of people that will be used in dataset.

In [5]:
def get_all_y(train_index):
    normal = list()
    fatigue = list()
    for epoch in range(num_epochs):
        normal.append(1)
    for epoch in range(num_epochs):
        fatigue.append(0)
    y = np.concatenate((normal, fatigue), axis=0)
            
    for person in train_index:
        if person == train_index[0]:
            y_people = y
        else:
            y_people = np.concatenate((y_people, y), axis=0)
            
    return y_people

Create subsets for multiple entropy.

In [6]:
def create_subset(arr, r):
    return list(combinations(arr, r))

Split dataset 1:1 for training and testing.

In [7]:
people_indexes = [i for i in range(num_people)]

X = get_all_X(train_index=people_indexes)
y = get_all_y(train_index=people_indexes)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

## MLP

In [8]:
clf = MLPClassifier(activation="logistic", hidden_layer_sizes=20, max_iter=500)
clf.fit(X_train, y_train)
clf.predict(X_test)
score = clf.score(X_test, y_test)
print(f"MLP score: {score}")

MLP score: 0.9730555555555556


/home/satan/.local/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


## Random forest

In [9]:
clf = RandomForestClassifier(n_estimators=500, max_features=22)
clf.fit(X_train, y_train)
clf.predict(X_test)
score = clf.score(X_test, y_test)
print(f"RandomForest score: {score}")

RandomForest score: 0.99


## K-neighbors

In [10]:
neigh = KNeighborsClassifier()
neigh.fit(X_train, y_train)
neigh.predict(X_test)
score = neigh.score(X_test, y_test)
print(f"K-neighbors score: {score}")

K-neighbors score: 0.9552777777777778


## SVM

### Using single entropy

In [11]:
for entropy in range(num_entropies):
    for person in range(num_people):
        normal_X = np.zeros(shape=(num_epochs, num_channels))
        for ch in range(num_channels):
            for epoch in range(num_epochs):
                normal_X[epoch][ch] = (ENTROPIES[entropy][0][person][epoch][ch][0])
        fatigue_X = np.zeros(shape=(num_epochs, num_channels))
        for ch in range(num_channels):
            for epoch in range(num_epochs):
                fatigue_X[epoch][ch] = (ENTROPIES[entropy][1][person][epoch][ch][0])

        # scale
        scaler = MinMaxScaler((-1, 1))
        normal_scaled_X = scaler.fit_transform(normal_X)
        fatigue_scaled_X = scaler.fit_transform(fatigue_X)
        
        X = np.concatenate((normal_scaled_X, fatigue_scaled_X), axis=0)
        if person == 0:
            matrix_all_X = X
        else:
            matrix_all_X = np.concatenate((matrix_all_X, X), axis=0)
           
    X = np.array(matrix_all_X)
    # train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    
    # train model
    svc = SVC(gamma=2**(-5), C=2**(-1), kernel="rbf")
    model = svc.fit(X_train, y_train)
    h = model.predict(X_test)
    score = accuracy_score(y_true=y_test, y_pred=h)
    clfrep_score = clfrep(y_true=y_test, y_pred=h)
    
    print(f"Entrpy: {entropy} Score: {score}")
    print(clfrep_score)

Entrpy: 0 Score: 0.7736111111111111
              precision    recall  f1-score   support

           0       0.75      0.83      0.79      1812
           1       0.81      0.71      0.76      1788

    accuracy                           0.77      3600
   macro avg       0.78      0.77      0.77      3600
weighted avg       0.78      0.77      0.77      3600

Entrpy: 1 Score: 0.8733333333333333
              precision    recall  f1-score   support

           0       0.86      0.89      0.88      1812
           1       0.88      0.86      0.87      1788

    accuracy                           0.87      3600
   macro avg       0.87      0.87      0.87      3600
weighted avg       0.87      0.87      0.87      3600

Entrpy: 2 Score: 0.7569444444444444
              precision    recall  f1-score   support

           0       0.74      0.80      0.77      1812
           1       0.78      0.71      0.74      1788

    accuracy                           0.76      3600
   macro avg       0

### Using two types of entropies

In [12]:
arr = [i for i in range(num_entropies)]
r = 2

ENTROPY_COMBINATIONS_2 = create_subset(arr, r)
n_ent = len(ENTROPY_COMBINATIONS_2)
ENT_X_2 = np.zeros(shape=(6, 2 * num_epochs, 2 * num_channels))


for person in range(num_people):
    i = 0
    for entropy in ENTROPY_COMBINATIONS_2:  
        normal_X1 = np.zeros(shape=(num_epochs, num_channels))    
        normal_X2 = np.zeros(shape=(num_epochs, num_channels))        
        for ch in range(num_channels):
            for epoch in range(num_epochs):
                normal_X1[epoch][ch] = (ENTROPIES[entropy[0]][0][person][epoch][ch][0])
                normal_X2[epoch][ch] = (ENTROPIES[entropy[1]][0][person][epoch][ch][0])                
        fatigue_X1 = np.zeros(shape=(num_epochs, num_channels))
        fatigue_X2 = np.zeros(shape=(num_epochs, num_channels))
        for ch in range(num_channels):
            for epoch in range(num_epochs):
                fatigue_X1[epoch][ch] = (ENTROPIES[entropy[0]][1][person][epoch][ch][0])
                fatigue_X2[epoch][ch] = (ENTROPIES[entropy[1]][1][person][epoch][ch][0])
      

        scaler = MinMaxScaler((-1, 1))
        normal_scaled_X1 = scaler.fit_transform(normal_X1)
        normal_scaled_X2 = scaler.fit_transform(normal_X2)   
        fatigue_scaled_X1 = scaler.fit_transform(fatigue_X1)
        fatigue_scaled_X2 = scaler.fit_transform(fatigue_X2)

        X_1 = np.concatenate((normal_scaled_X1, fatigue_scaled_X1), axis=0)
        X_2 = np.concatenate((normal_scaled_X2, fatigue_scaled_X2), axis=0)
        
        X = np.concatenate((X_1, X_2), axis=1)
        ENT_X_2[i] = X
        i += 1
    
    for i in range(0, n_ent):
        if person == 0 and i == 0:
            ENT_X_2_ALL = ENT_X_2[i]
        else:
            ENT_X_2_ALL = np.concatenate((ENT_X_2_ALL, ENT_X_2[i]), axis=0)


X = np.array(ENT_X_2_ALL)

y_2 = []
for person in range(n_ent):
    y_2 = np.concatenate((y_2, y), axis=0)
y_2 = np.array(y_2)


for entropy in range(n_ent):
    # train model
    X_train, X_test, y_train, y_test = train_test_split(X, y_2, test_size=0.5, random_state=0)
    svc = SVC(gamma=2**(-5), C=2**(-1), kernel="rbf")
    model = svc.fit(X_train, y_train)
    h = model.predict(X_test)
    score = accuracy_score(y_true=y_test, y_pred=h)
    clfrep_score = clfrep(y_true=y_test, y_pred=h)
    
    print(f"Entrpy: {entropy} Score: {score}")
    print(clfrep_score)

Entrpy: 0 Score: 0.9058333333333334
              precision    recall  f1-score   support

         0.0       0.90      0.92      0.91     10685
         1.0       0.92      0.90      0.91     10915

    accuracy                           0.91     21600
   macro avg       0.91      0.91      0.91     21600
weighted avg       0.91      0.91      0.91     21600

Entrpy: 1 Score: 0.9058333333333334
              precision    recall  f1-score   support

         0.0       0.90      0.92      0.91     10685
         1.0       0.92      0.90      0.91     10915

    accuracy                           0.91     21600
   macro avg       0.91      0.91      0.91     21600
weighted avg       0.91      0.91      0.91     21600

Entrpy: 2 Score: 0.9058333333333334
              precision    recall  f1-score   support

         0.0       0.90      0.92      0.91     10685
         1.0       0.92      0.90      0.91     10915

    accuracy                           0.91     21600
   macro avg       0

### Using three types of entropies

In [13]:
r = 3

ENTROPY_COMBINATIONS_3 = create_subset(arr, r)
n_ent = len(ENTROPY_COMBINATIONS_3)
ENT_X_3 = np.zeros(shape=(6, 2 * num_epochs, 3 * num_channels))


for person in range(12):
    i = 0
    for entropy in ENTROPY_COMBINATIONS_3:  
        normal_X1 = np.zeros(shape=(num_epochs, num_channels))    
        normal_X2 = np.zeros(shape=(num_epochs, num_channels))
        normal_X3 = np.zeros(shape=(num_epochs, num_channels))    
        for ch in range(num_channels):
            for epoch in range(num_epochs):
                normal_X1[epoch][ch] = (ENTROPIES[entropy[0]][0][person][epoch][ch][0])
                normal_X2[epoch][ch] = (ENTROPIES[entropy[1]][0][person][epoch][ch][0])
                normal_X3[epoch][ch] = (ENTROPIES[entropy[2]][0][person][epoch][ch][0])
        fatigue_X1 = np.zeros(shape=(num_epochs, num_channels))
        fatigue_X2 = np.zeros(shape=(num_epochs, num_channels))
        fatigue_X3 = np.zeros(shape=(num_epochs, num_channels))
        for ch in range(num_channels):
            for epoch in range(num_epochs):
                fatigue_X1[epoch][ch] = (ENTROPIES[entropy[0]][1][person][epoch][ch][0])
                fatigue_X2[epoch][ch] = (ENTROPIES[entropy[1]][1][person][epoch][ch][0])
                fatigue_X3[epoch][ch] = (ENTROPIES[entropy[2]][1][person][epoch][ch][0])

        scaler = MinMaxScaler((-1, 1))
        normal_scaled_X1 = scaler.fit_transform(normal_X1)
        normal_scaled_X2 = scaler.fit_transform(normal_X2)
        normal_scaled_X3 = scaler.fit_transform(normal_X3)       
        fatigue_scaled_X1 = scaler.fit_transform(fatigue_X1)
        fatigue_scaled_X2 = scaler.fit_transform(fatigue_X2)
        fatigue_scaled_X3 = scaler.fit_transform(fatigue_X3)

        X_1 = np.concatenate((normal_scaled_X1, fatigue_scaled_X1), axis=0)
        X_2 = np.concatenate((normal_scaled_X2, fatigue_scaled_X2), axis=0)
        X_3 = np.concatenate((normal_scaled_X3, fatigue_scaled_X3), axis=0)
        
        X = np.concatenate((X_1, X_2, X_3),axis=1)
        
        ENT_X_3[i] = X
        i += 1
        
    for i in range(n_ent):
        if person == 0 and i == 0:
            ENT_X_3_ALL = ENT_X_3[i]
        else:
            ENT_X_3_ALL = np.concatenate((ENT_X_3_ALL, ENT_X_3[i]), axis=0)
        
        
X = np.array(ENT_X_3_ALL)

y_3 = []
for person in range(n_ent):
    y_3 = np.concatenate((y_3, y), axis=0)
y_3 = np.array(y_3)


for entropy in range(n_ent):
    X_train, X_test, y_train, y_test = train_test_split(X, y_3, test_size=0.5, random_state=0)
    svc = SVC(gamma=2**(-5), C=2**(-1), kernel="rbf")
    model = svc.fit(X_train, y_train)
    h = model.predict(X_test)
    score = accuracy_score(y_true=y_test, y_pred=h)
    clfrep_score = clfrep(y_true=y_test, y_pred=h)
    
    print(f"Entrpy: {entropy} Score: {score}")
    print(clfrep_score)

Entrpy: 0 Score: 0.9449305555555556
              precision    recall  f1-score   support

         0.0       0.93      0.96      0.95      7168
         1.0       0.96      0.93      0.94      7232

    accuracy                           0.94     14400
   macro avg       0.95      0.95      0.94     14400
weighted avg       0.95      0.94      0.94     14400

Entrpy: 1 Score: 0.9449305555555556
              precision    recall  f1-score   support

         0.0       0.93      0.96      0.95      7168
         1.0       0.96      0.93      0.94      7232

    accuracy                           0.94     14400
   macro avg       0.95      0.95      0.94     14400
weighted avg       0.95      0.94      0.94     14400

Entrpy: 2 Score: 0.9449305555555556
              precision    recall  f1-score   support

         0.0       0.93      0.96      0.95      7168
         1.0       0.96      0.93      0.94      7232

    accuracy                           0.94     14400
   macro avg       0

### Using all four entropies

#### Grid search

In [14]:
people_indexes = [i for i in range(num_people)]

X = get_all_X(train_index=people_indexes)


C_list = [2**(-5), 2**(-1), 2**(5)]
gamma_list = [2**(-10), 2**(-5), 2**(10)]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

grid = 0
for i in C_list:
    for j in gamma_list:
        svc = SVC(gamma=j, C=i, kernel="rbf")
        model = svc.fit(X_train, y_train)
        h = model.predict(X_test)
        score = accuracy_score(y_true=y_test, y_pred=h)
        clfrep_score = clfrep(y_true=y_test, y_pred=h)
        grid += 1
        
        print(f"score: C = {i} gamma = {j} : {score}")
        print(clfrep_score)

score: C = 0.03125 gamma = 0.0009765625 : 0.5041666666666667
              precision    recall  f1-score   support

           0       0.94      0.02      0.03      1812
           1       0.50      1.00      0.67      1788

    accuracy                           0.50      3600
   macro avg       0.72      0.51      0.35      3600
weighted avg       0.72      0.50      0.35      3600

score: C = 0.03125 gamma = 0.03125 : 0.7736111111111111
              precision    recall  f1-score   support

           0       0.74      0.84      0.79      1812
           1       0.81      0.70      0.76      1788

    accuracy                           0.77      3600
   macro avg       0.78      0.77      0.77      3600
weighted avg       0.78      0.77      0.77      3600



/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


score: C = 0.03125 gamma = 1024 : 0.49666666666666665
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1812
           1       0.50      1.00      0.66      1788

    accuracy                           0.50      3600
   macro avg       0.25      0.50      0.33      3600
weighted avg       0.25      0.50      0.33      3600

score: C = 0.5 gamma = 0.0009765625 : 0.7097222222222223
              precision    recall  f1-score   support

           0       0.66      0.87      0.75      1812
           1       0.80      0.55      0.65      1788

    accuracy                           0.71      3600
   macro avg       0.73      0.71      0.70      3600
weighted avg       0.73      0.71      0.70      3600

score: C = 0.5 gamma = 0.03125 : 0.9530555555555555
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      1812
           1       0.96      0.94      0.95      1788

    accuracy       

/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


score: C = 0.5 gamma = 1024 : 0.49666666666666665
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1812
           1       0.50      1.00      0.66      1788

    accuracy                           0.50      3600
   macro avg       0.25      0.50      0.33      3600
weighted avg       0.25      0.50      0.33      3600

score: C = 32 gamma = 0.0009765625 : 0.8919444444444444
              precision    recall  f1-score   support

           0       0.88      0.92      0.90      1812
           1       0.91      0.87      0.89      1788

    accuracy                           0.89      3600
   macro avg       0.89      0.89      0.89      3600
weighted avg       0.89      0.89      0.89      3600

score: C = 32 gamma = 0.03125 : 0.9855555555555555
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      1812
           1       0.99      0.98      0.99      1788

    accuracy             

/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Leave-one-out (LOO)

In [ ]:
people_indexes = [i for i in range(num_people)]

grid = 0
for i in C_list:
    for j in gamma_list:
        loo = LeaveOneOut()
        for train_index, test_index in loo.split(people_indexes):
            X_train = get_all_X(train_index)
            X_test= get_all_X(test_index)
            y_train = get_all_y(train_index)
            y_test = get_all_y(test_index)
            
            svc = SVC(gamma=j, C=i, kernel="rbf")
            model = svc.fit(X_train, y_train)
            h = model.predict(X_test)
            score = accuracy_score(y_true=y_test, y_pred=h)
            clfrep_score = clfrep(y_true=y_test, y_pred=h)
            
            print(f"score: person: {test_index} C = {i} gamma = {j} : {score}")
            print(clfrep_score)

score: person: [0] C = 0.03125 gamma = 0.0009765625 : 0.49
              precision    recall  f1-score   support

           0       0.47      0.13      0.21       300
           1       0.49      0.85      0.62       300

    accuracy                           0.49       600
   macro avg       0.48      0.49      0.42       600
weighted avg       0.48      0.49      0.42       600

score: person: [1] C = 0.03125 gamma = 0.0009765625 : 0.69
              precision    recall  f1-score   support

           0       0.90      0.43      0.58       300
           1       0.62      0.95      0.75       300

    accuracy                           0.69       600
   macro avg       0.76      0.69      0.67       600
weighted avg       0.76      0.69      0.67       600

score: person: [2] C = 0.03125 gamma = 0.0009765625 : 0.465
              precision    recall  f1-score   support

           0       0.41      0.16      0.23       300
           1       0.48      0.77      0.59       300

    

score: person: [9] C = 0.03125 gamma = 0.03125 : 0.5183333333333333
              precision    recall  f1-score   support

           0       0.51      0.97      0.67       300
           1       0.69      0.07      0.12       300

    accuracy                           0.52       600
   macro avg       0.60      0.52      0.39       600
weighted avg       0.60      0.52      0.39       600

score: person: [10] C = 0.03125 gamma = 0.03125 : 0.27166666666666667
              precision    recall  f1-score   support

           0       0.35      0.51      0.41       300
           1       0.06      0.03      0.04       300

    accuracy                           0.27       600
   macro avg       0.20      0.27      0.23       600
weighted avg       0.20      0.27      0.23       600

score: person: [11] C = 0.03125 gamma = 0.03125 : 0.225
              precision    recall  f1-score   support

           0       0.30      0.42      0.35       300
           1       0.04      0.03      0.03

/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


score: person: [0] C = 0.03125 gamma = 1024 : 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       300
           1       0.50      1.00      0.67       300

    accuracy                           0.50       600
   macro avg       0.25      0.50      0.33       600
weighted avg       0.25      0.50      0.33       600



/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

score: person: [9] C = 0.03125 gamma = 1024 : 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       300
           1       0.50      1.00      0.67       300

    accuracy                           0.50       600
   macro avg       0.25      0.50      0.33       600
weighted avg       0.25      0.50      0.33       600



/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


score: person: [10] C = 0.03125 gamma = 1024 : 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       300
           1       0.50      1.00      0.67       300

    accuracy                           0.50       600
   macro avg       0.25      0.50      0.33       600
weighted avg       0.25      0.50      0.33       600



/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/satan/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


score: person: [11] C = 0.03125 gamma = 1024 : 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       300
           1       0.50      1.00      0.67       300

    accuracy                           0.50       600
   macro avg       0.25      0.50      0.33       600
weighted avg       0.25      0.50      0.33       600

score: person: [0] C = 0.5 gamma = 0.0009765625 : 0.44
              precision    recall  f1-score   support

           0       0.41      0.26      0.32       300
           1       0.46      0.62      0.53       300

    accuracy                           0.44       600
   macro avg       0.43      0.44      0.42       600
weighted avg       0.43      0.44      0.42       600

score: person: [1] C = 0.5 gamma = 0.0009765625 : 0.6483333333333333
              precision    recall  f1-score   support

           0       0.71      0.50      0.59       300
           1       0.61      0.79      0.69       300

    acc

In paper the SVM classifier reached its optimized status with c = -1 and g = -5.